In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("models/embeddinggemma-300m", trust_remote_code=True)

/home/prxneeth/miniconda3/envs/ghci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df = pd.read_csv("business_transactions_dataset.csv")
df

,Unnamed: 0,name,fsq_category_ids,Count,category_label,category_array,transaction_string,seed
0,0,Western Union,63be6904847c3692a84b9b3d,5993,Business and Professional Services > Financial...,"['Business and Professional Services', 'Financ...",WESTERN UNION #45678 SEND $29.99 TO CHICAGO IL...,1127428596
1,1,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032,Business and Professional Services > Financial...,"['Business and Professional Services', 'Financ...",The PNC Financial Services Group #7890 ACCOUNT...,292935982
2,2,Starbucks,4bf58dd8d48988d1e0931735,4893,"Dining and Drinking > Cafe, Coffee, and Tea Ho...","['Dining and Drinking', 'Cafe, Coffee, and Tea...",STARBUCKS COFFEE #98765 STORE 98765 PURCHASE $...,1047754538
3,3,Blue Rhino,63be6904847c3692a84b9b34,4818,Business and Professional Services > Chemicals...,"['Business and Professional Services', 'Chemic...","BLUE RHINO #4567 PURCHASE $1,250.00 FOR LABORA...",1537448773
4,4,Redbox,4bf58dd8d48988d126951735,4718,Retail > Video Store,"['Retail', 'Video Store']",Redbox #4567 DVD RENTAL $12.99 02/03/25 AZ,1248492378
...,...,...,...,...,...,...,...,...
17466,17467,American Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"['Retail', 'Furniture and Home Store', 'Home A...",AMERICAN APPLIANCE REPAIR #7890 SERVICE CALL 1...,900410143
17467,17468,A-1 Appliance Service,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"['Retail', 'Furniture and Home Store', 'Home A...",A-1 Appliance Service #4567 SERVICE CALL 2025-...,957908134
17468,17469,Affordable Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"['Retail', 'Furniture and Home Store', 'Home A...",AFFORDABLE APPLIANCE REPAIR #4567 SERVICE CALL...,935033619
17469,17470,NightLight Pediatric Urgent Care,4bf58dd8d48988d194941735,5,Health and Medicine > Emergency Service > Emer...,"['Health and Medicine', 'Emergency Service', '...",NIGHTLIGHT PEDIATRIC URGENT CARE #789 VISIT 11...,359454038


In [3]:
import pandas as pd
import numpy as np

batch_size = 64
# 1. Encode Transactions (Batch Processing)
# Instead of one by one, we send the whole list. Sentence-transformers handles batching internally.
print("Encoding Transactions...")
transaction_embeddings = model.encode(
    df["transaction_string"].tolist(), 
    batch_size=batch_size, 
    show_progress_bar=True, 
    convert_to_numpy=True
)

# Assign back to DataFrame (converting numpy array of arrays to a list of arrays for the column)
df["transaction_embeddings"] = list(transaction_embeddings)

# 2. Encode Categories (Deduplication Strategy)
# Since categories repeat across rows, we only encode UNIQUE hierarchical paths.
print("Processing Categories...")

def make_hierarchy(cats):
    # Faster string construction
    if not cats: return []
    res = []
    current_path = ""
    for c in cats:
        current_path = f"{current_path} > {c}" if current_path else c
        res.append(current_path)
    return res

# A. Generate hierarchical strings for all rows (CPU bound, fast)
# We store these temporarily to find unique values
temp_hierarchies = df["category_array"].apply(make_hierarchy)

# B. Find unique strings to encode (massive reduction in compute time)
# Flatten the list of lists to find all unique hierarchical strings
all_flat_cats = [cat for row in temp_hierarchies for cat in row]
unique_cats = list(set(all_flat_cats))

print(f"Encoding {len(unique_cats)} unique category paths...")
unique_cat_embeddings = model.encode(
    unique_cats, 
    batch_size=batch_size, 
    show_progress_bar=True
)

# C. Create a lookup dictionary
cat_lookup = dict(zip(unique_cats, unique_cat_embeddings))

# D. Map embeddings back to the dataframe
df["category_embeddings"] = temp_hierarchies.apply(
    lambda row: [cat_lookup[x] for x in row]
)

print("Done.")


Encoding Transactions...


Batches: 100%|██████████| 273/273 [00:42<00:00,  6.38it/s]


Processing Categories...
Encoding 11433 unique category paths...


Batches: 100%|██████████| 179/179 [00:59<00:00,  3.02it/s]

Done.


In [4]:
import numpy as np

# 1. Prepare Transaction Embeddings (Fixed shape)
# Stack ensures it's a proper 2D matrix (N, 768)
txn_matrix = np.vstack(df["transaction_embeddings"].values)

# 2. Prepare Category Embeddings (Variable shape -> Flattened + Offsets)
# We flatten the list of lists into one giant matrix
flat_cat_embeds = np.vstack([np.vstack(x) for x in df["category_embeddings"] if len(x) > 0])

# We calculate where each row starts and ends
# Calculate lengths of each row
lengths = df["category_embeddings"].apply(len).to_numpy()
# Calculate cumulative offsets (start indices)
offsets = np.insert(np.cumsum(lengths), 0, 0)

# 3. Save to disk
np.save("dataset/gemma/txn_embeds_mlm.npy", txn_matrix)
np.save("dataset/gemma/cat_embeds_flat_mlm.npy", flat_cat_embeds)
np.save("dataset/gemma/cat_offsets_mlm.npy", offsets)
np.save("dataset/gemma/cat_lengths_mlm.npy", lengths) # Optional, but helpful

print("Saved as numpy arrays.")

Saved as numpy arrays.


# You generally need to ensure columns are lists, not np.arrays, before saving to parquet
df.to_parquet("embeddings.parquet", engine="pyarrow")